In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torchvision as T
import torch
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import math

/Users/paul/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/paul/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <8080486D-E510-3000-AA6A-F3AD49ACC172> /Users/paul/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <EB47298B-CC51-3FA7-885A-620F458D20C5> /Users/paul/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
data = pd.read_csv("/Users/paul/Downloads/data_match_clean(1).csv", delimiter = ";", index_col = 1)
data = data.drop(["Unnamed: 0"], axis = 1)
data.head()

,schedule_time,schedule_round,schedule_day,result_home,gf_home,gf_away,xg_home,xg_away,attendance,captain_home,...,misc_performance_crdr_away,misc_performance_2crdy_away,misc_performance_fls_away,misc_performance_fld_away,misc_performance_off_away,misc_performance_og_away,misc_performance_recov_away,misc_aerialduels_won_away,misc_aerialduels_lost_away,misc_aerialduels_won_perc_away
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-19,18:00,Matchweek 1,Sat,W,3.0,0.0,1.1,1.4,39226.0,Gianluigi Buffon,...,0.0,0.0,11.0,13.0,0.0,0.0,40.0,9.0,8.0,52.9
2017-09-09,18:00,Matchweek 3,Sat,W,3.0,0.0,1.3,0.6,39457.0,Stephan Lichtsteiner,...,0.0,0.0,7.0,15.0,1.0,1.0,34.0,10.0,10.0,50.0
2017-09-20,20:45,Matchweek 5,Wed,W,1.0,0.0,0.9,0.4,35652.0,Andrea Barzagli,...,1.0,1.0,23.0,10.0,1.0,0.0,46.0,8.0,9.0,47.1
2017-09-23,20:45,Matchweek 6,Sat,W,4.0,0.0,2.2,0.2,40856.0,Gianluigi Buffon,...,1.0,1.0,12.0,16.0,1.0,0.0,41.0,6.0,11.0,35.3
2017-10-14,18:00,Matchweek 8,Sat,L,1.0,2.0,2.3,1.6,39145.0,Gianluigi Buffon,...,0.0,0.0,14.0,8.0,1.0,0.0,47.0,14.0,12.0,53.8


In [4]:
class preprocess():
    
    def __init__(self, data_frame):
        self.data_frame = data_frame
        
        self.day_dict = {}
        i = 0
        for day in self.data_frame.schedule_day.unique():
            self.day_dict[day] = i
            i += 1
        
        self.result = {}
        i = 0
        for res in self.data_frame.result_home.unique():
            self.result[res] = i
            i += 1
        #print(self.result)
        
        self.capt = {}
        i = 0
        for cap in set(list(self.data_frame.captain_home) + list(self.data_frame.captain_away)):
            self.capt[cap] = i
            i += 1
            
        self.formation = {}
        i = 0
        for form in self.data_frame.formation_home.unique():
            self.formation[form] = i
            i += 1
            
        self.referee = {}
        i = 0
        for ref in self.data_frame.referee.unique():
            self.referee[ref] = i
            i += 1
        
        self.teams = {}
        i = 0
        for team in set(list(self.data_frame.fbref_home_id) + list(self.data_frame.fbref_away_id)):
            self.teams[team] = i
            i += 1
        #print(self.teams)
        
        self.season = {}
        i = 0
        for seas in self.data_frame.fbref_season.unique():
            self.season[seas] = i
            i += 1
        
        self.time = {}
        i = 0
        for tim in self.data_frame.schedule_time.unique():
            self.time[tim] = i
            i += 1
            
        self.ligue = {}
        i = 0
        for lig in self.data_frame.fbref_league_id.unique():
            self.ligue[lig] = i
            i += 1
            
        self.match = {}
        i = 0
        for mat in self.data_frame.fbref_match_id.unique():
            self.match[mat] = i
            i += 1
        
        self.data_frame.schedule_day.replace(self.day_dict, inplace = True)
        self.data_frame.result_home.replace(self.result, inplace = True)
        self.data_frame.result_away.replace(self.result, inplace = True)
        self.data_frame.captain_home.replace(self.capt, inplace = True)
        self.data_frame.captain_away.replace(self.capt, inplace = True)
        self.data_frame.formation_home.replace(self.formation, inplace = True)
        self.data_frame.formation_away.replace(self.formation, inplace = True)
        self.data_frame.referee.replace(self.referee, inplace = True)
        self.data_frame.fbref_home_id.replace(self.teams, inplace = True)
        self.data_frame.fbref_away_id.replace(self.teams, inplace = True)
        self.data_frame.fbref_season.replace(self.season, inplace = True)
        self.data_frame.schedule_time.replace(self.time, inplace = True)
        self.data_frame.fbref_league_id.replace(self.ligue, inplace = True)
        self.data_frame.fbref_match_id.replace(self.match, inplace = True)
        
        self.features = ['shooting_standard_gls', 'shooting_standard_sh', 'shooting_standard_sot', 
                         'shooting_standard_sot_perc', 'shooting_standard_g_per_sh', 'shooting_standard_g_per_sot',
                         'shooting_standard_dist', 'shooting_standard_fk', 'shooting_standard_pk', 
                         'shooting_standard_pkatt', 'shooting_expected_npxg', 'shooting_expected_npxg_per_sh',
                         'shooting_expected_g_minus_xg', 'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
                         'keeper_performance_saves', 'keeper_performance_save_perc', 'keeper_performance_cs',
                         'keeper_performance_psxg', 'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt', 
                         'keeper_penaltykicks_pksv', 'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
                         'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr', 'keeper_passes_launch_perc',
                         'keeper_passes_avglen', 'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc', 
                         'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp', 'keeper_crosses_stp_perc',
                         'keeper_sweeper_number_opa', 'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
                         'passing_total_cmp_perc', 'passing_total_totdist', 'passing_total_prgdist', 'passing_short_cmp',
                         'passing_short_att', 'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
                         'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att', 'passing_long_cmp_perc',
                         'passing_attacking_ast', 'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
                         'passing_attacking_1_per_3', 'passing_attacking_ppa', 'passing_attacking_crspa', 'passing_attacking_prgp',
                         'passing_types_passtypes_live', 'passing_types_passtypes_dead', 'passing_types_passtypes_fk',
                         'passing_types_passtypes_tb', 'passing_types_passtypes_sw', 'passing_types_passtypes_crs',
                         'passing_types_passtypes_ti', 'passing_types_passtypes_ck', 'passing_types_cornerkicks_in',
                         'passing_types_cornerkicks_out', 'passing_types_cornerkicks_str', 'passing_types_outcomes_off',
                         'passing_types_outcomes_blocks', 'gca_scatypes_sca', 'gca_scatypes_passlive', 'gca_scatypes_passdead',
                         'gca_scatypes_to', 'gca_scatypes_sh', 'gca_scatypes_fld', 'gca_scatypes_def', 'gca_gcatypes_gca', 
                         'gca_gcatypes_passlive', 'gca_gcatypes_passdead', 'gca_gcatypes_to', 'gca_gcatypes_sh',
                         'gca_gcatypes_fld',  'gca_gcatypes_def', 'defense_tackles_tkl', 'defense_tackles_tklw',
                         'defense_tackles_def3rd', 'defense_tackles_mid3rd', 'defense_tackles_att3rd', 'defense_challenges_tkl',
                         'defense_challenges_att', 'defense_challenges_tkl_perc', 'defense_challenges_lost', 'defense_blocks_blocks',
                         'defense_blocks_sh', 'defense_blocks_pass', 'defense_general_int', 'defense_general_tkl_plus_int',
                         'defense_general_clr', 'defense_general_err', 'possession_general_poss', 'possession_touches_touches',
                         'possession_touches_defpen', 'possession_touches_def3rd', 'possession_touches_mid3rd',
                         'possession_touches_att3rd', 'possession_touches_attpen', 'possession_touches_live', 'possession_takeons_att',
                         'possession_takeons_succ', 'possession_takeons_succ_perc', 'possession_takeons_tkld', 'possession_takeons_tkld_perc',
                         'possession_carries_carries', 'possession_carries_totdist', 'possession_carries_prgdist',
                         'possession_carries_prgc', 'possession_carries_1_per_3', 'possession_carries_cpa', 'possession_carries_mis',
                         'possession_carries_dis', 'possession_receiving_rec', 'possession_receiving_prgr',
                         'misc_performance_crdy', 'misc_performance_crdr', 'misc_performance_2crdy', 'misc_performance_fls',
                         'misc_performance_fld', 'misc_performance_off', 'misc_performance_og', 'misc_performance_recov',
                         'misc_aerialduels_won', 'misc_aerialduels_lost', 'misc_aerialduels_won_perc', 'keeper_penaltykicks_pka',
                         'attendance']
        
        
        #self.data_frame = data_frame
    
    def data_frame(self):
        return self.data_frame
        
    def dataset_team(self, team_id):
        # split dataset 
        random_team1 = self.data_frame[self.data_frame.fbref_home_id == team_id].loc[:, self.data_frame.columns.str.endswith('home')]
        random_team2 = self.data_frame[self.data_frame.fbref_away_id == team_id].loc[:, self.data_frame.columns.str.endswith('away')]
        random_team3 = self.data_frame[(self.data_frame.fbref_home_id == team_id) |
                        (self.data_frame.fbref_away_id == team_id)].loc[:,['schedule_time', 'attendance',
                                                                           'referee', 'fbref_season',
                                                                           'fbref_league_id', 'fbref_home_id',
                                                                           'fbref_away_id', 'fbref_match_id']]

        #random_team2.rename({"gf_away": "gf_home", "gf_home": "gf_away"}, axis = 1, inplace = True)
        # preprocess chunks
        self.concat_preprocess(random_team1, "home")
        self.concat_preprocess(random_team2, "away")
        random_team = pd.concat([random_team1, random_team2])

        # dummy to account for home and away
        random_team3["home"] = np.where(random_team3.fbref_home_id == team_id, 1, 0)

        # rename columns to change them
        random_team3_away = random_team3.loc[random_team3.fbref_away_id == team_id].copy()
        random_team3_away.rename({"fbref_home_id": "fbref_away_id",
                                  "fbref_away_id": "fbref_home_id"}, axis = 1, inplace = True)
        random_team3_home = random_team3[random_team3.fbref_home_id == team_id]

        # concat after changing column names and rename 
        random_team3 = pd.concat([random_team3_home, random_team3_away])
        random_team3.rename({"fbref_home_id": "fbref_own", 
                             "fbref_away_id": "fbref_oppon",
                             "gf_home": "gf_own", 
                             "gf_away": "gf_own"}, axis = 1, inplace = True)

        self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()

        return self.end_team

    def concat_preprocess(self, df, where):
        colname_dict = {}
        result_dict = {self.result["W"]: "L", self.result["D"]: "D", self.result["L"]: "W"}
        for colname in self.data_frame.loc[:, self.data_frame.columns.str.endswith(where)]:
            colname_dict[colname]  = colname[:-5]
    
        return df.rename(colname_dict, axis = 1, inplace = True)
    
    def return_dicts(self, dict_name):
        if dict_name == "day":
            return self.day_dict
        elif dict_name == "result":
            return self.result
        elif dict_name == "captain":
            return self.capt
        elif dict_name == "formation":
            return self.formation
        elif dict_name == "referee":
            return self.referee
        elif dict_name == "teams":
            return self.teams
        elif dict_name == "season":
            return self.season
        
    def data_for_team1(self, date, team_1, window_size = 10, discount = 0.5):
        
        object = StandardScaler()
        dataset1 = self.dataset_team(team_1)
        cols_for_movavg = dataset1.columns.isin(self.features)
        dataset1.loc[:,cols_for_movavg] = object.fit_transform(dataset1.loc[:,cols_for_movavg])#self.moving_average(dataset1.loc[:,cols_for_movavg], window_size, discount))
        dataset1 = dataset1.add_suffix('_1')
        dataset1 = dataset1.loc[date]
        
        dataset2 = self.dataset_team(dataset1.fbref_oppon_1)
        dataset2.loc[:,cols_for_movavg] = object.fit_transform(#self.moving_average(dataset2.loc[:,cols_for_movavg], window_size, discount))
        dataset2 = dataset2.add_suffix('_2')
        dataset2 = dataset2.loc[date]
        
        cols_to_drop = ['gf_1', 'xg_1', 'schedule_time_1', 'referee_1', 'fbref_season_1', 'fbref_league_id_1', 
                'fbref_own_1', 'fbref_oppon_1', 'fbref_match_id_1','gf_2', 'xg_2', 
                'fbref_own_2', 'fbref_oppon_2', 'fbref_match_id_2']
        
        to_return = pd.concat([dataset1, dataset2])#.loc[date]
        to_return = to_return[~to_return.index.isin(cols_to_drop)]
        
        X = to_return[~to_return.index.isin(["result_1", "result_2"])]
        y = to_return[to_return.index.isin(["result_1"])]
        
        return X, y
    
    def moving_average(self, df, window_size = 10, discount = 0.5, date = None):
        "compute the moving average with recency bias"
        weights = np.array([discount**x for x in reversed(range(window_size))]) # discount y_t-windowsize with discount**windowsize
        weights = np.append(weights , 0)
        sum_weights = np.sum(weights)
        mov_avg = df.rolling(window = window_size + 1).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False) # compute moving average of window_size 
        return mov_avg
        

In [5]:
new_data = preprocess(data)
new_data1 = new_data.data_frame#.dataset_team("e0652b02").captain.unique()

In [88]:
new_data.data_frame

,schedule_time,schedule_round,schedule_day,result_home,gf_home,gf_away,xg_home,xg_away,attendance,captain_home,...,misc_performance_crdr_away,misc_performance_2crdy_away,misc_performance_fls_away,misc_performance_fld_away,misc_performance_off_away,misc_performance_og_away,misc_performance_recov_away,misc_aerialduels_won_away,misc_aerialduels_lost_away,misc_aerialduels_won_perc_away
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-19,0,Matchweek 1,0,0,3.0,0.0,1.1,1.4,39226.0,29,...,0.0,0.0,11.0,13.0,0.0,0.0,40.0,9.0,8.0,52.9
2017-09-09,0,Matchweek 3,0,0,3.0,0.0,1.3,0.6,39457.0,861,...,0.0,0.0,7.0,15.0,1.0,1.0,34.0,10.0,10.0,50.0
2017-09-20,1,Matchweek 5,1,0,1.0,0.0,0.9,0.4,35652.0,135,...,1.0,1.0,23.0,10.0,1.0,0.0,46.0,8.0,9.0,47.1
2017-09-23,1,Matchweek 6,0,0,4.0,0.0,2.2,0.2,40856.0,29,...,1.0,1.0,12.0,16.0,1.0,0.0,41.0,6.0,11.0,35.3
2017-10-14,0,Matchweek 8,0,1,1.0,2.0,2.3,1.6,39145.0,29,...,0.0,0.0,14.0,8.0,1.0,0.0,47.0,14.0,12.0,53.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-15,12,Matchweek 7,1,1,0.0,2.0,0.5,1.9,30438.0,227,...,0.0,0.0,10.0,14.0,1.0,0.0,51.0,23.0,17.0,57.5
2023-03-18,2,Matchweek 28,0,2,3.0,3.0,2.4,2.3,30157.0,227,...,0.0,0.0,8.0,12.0,1.0,0.0,48.0,14.0,10.0,58.3
2023-04-08,34,Matchweek 30,0,1,1.0,4.0,0.4,2.9,30963.0,227,...,0.0,0.0,10.0,6.0,2.0,0.0,45.0,13.0,7.0,65.0


In [18]:
df = new_data.dataset_team(1)
df
cols_to_transform = ['shooting_standard_gls', 'shooting_standard_sh', 'shooting_standard_sot', 
                         'shooting_standard_sot_perc', 'shooting_standard_g_per_sh', 'shooting_standard_g_per_sot',
                         'shooting_standard_dist', 'shooting_standard_fk', 'shooting_standard_pk', 
                         'shooting_standard_pkatt', 'shooting_expected_npxg', 'shooting_expected_npxg_per_sh',
                         'shooting_expected_g_minus_xg', 'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
                         'keeper_performance_saves', 'keeper_performance_save_perc', 'keeper_performance_cs',
                         'keeper_performance_psxg', 'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt', 
                         'keeper_penaltykicks_pksv', 'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
                         'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr', 'keeper_passes_launch_perc',
                         'keeper_passes_avglen', 'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc', 
                         'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp', 'keeper_crosses_stp_perc',
                         'keeper_sweeper_number_opa', 'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
                         'passing_total_cmp_perc', 'passing_total_totdist', 'passing_total_prgdist', 'passing_short_cmp',
                         'passing_short_att', 'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
                         'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att', 'passing_long_cmp_perc',
                         'passing_attacking_ast', 'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
                         'passing_attacking_1_per_3', 'passing_attacking_ppa', 'passing_attacking_crspa', 'passing_attacking_prgp',
                         'passing_types_passtypes_live', 'passing_types_passtypes_dead', 'passing_types_passtypes_fk',
                         'passing_types_passtypes_tb', 'passing_types_passtypes_sw', 'passing_types_passtypes_crs',
                         'passing_types_passtypes_ti', 'passing_types_passtypes_ck', 'passing_types_cornerkicks_in',
                         'passing_types_cornerkicks_out', 'passing_types_cornerkicks_str', 'passing_types_outcomes_off',
                         'passing_types_outcomes_blocks', 'gca_scatypes_sca', 'gca_scatypes_passlive', 'gca_scatypes_passdead',
                         'gca_scatypes_to', 'gca_scatypes_sh', 'gca_scatypes_fld', 'gca_scatypes_def', 'gca_gcatypes_gca', 
                         'gca_gcatypes_passlive', 'gca_gcatypes_passdead', 'gca_gcatypes_to', 'gca_gcatypes_sh',
                         'gca_gcatypes_fld',  'gca_gcatypes_def', 'defense_tackles_tkl', 'defense_tackles_tklw',
                         'defense_tackles_def3rd', 'defense_tackles_mid3rd', 'defense_tackles_att3rd', 'defense_challenges_tkl',
                         'defense_challenges_att', 'defense_challenges_tkl_perc', 'defense_challenges_lost', 'defense_blocks_blocks',
                         'defense_blocks_sh', 'defense_blocks_pass', 'defense_general_int', 'defense_general_tkl_plus_int',
                         'defense_general_clr', 'defense_general_err', 'possession_general_poss', 'possession_touches_touches',
                         'possession_touches_defpen', 'possession_touches_def3rd', 'possession_touches_mid3rd',
                         'possession_touches_att3rd', 'possession_touches_attpen', 'possession_touches_live', 'possession_takeons_att',
                         'possession_takeons_succ', 'possession_takeons_succ_perc', 'possession_takeons_tkld', 'possession_takeons_tkld_perc',
                         'possession_carries_carries', 'possession_carries_totdist', 'possession_carries_prgdist',
                         'possession_carries_prgc', 'possession_carries_1_per_3', 'possession_carries_cpa', 'possession_carries_mis',
                         'possession_carries_dis', 'possession_receiving_rec', 'possession_receiving_prgr',
                         'misc_performance_crdy', 'misc_performance_crdr', 'misc_performance_2crdy', 'misc_performance_fls',
                         'misc_performance_fld', 'misc_performance_off', 'misc_performance_og', 'misc_performance_recov',
                         'misc_aerialduels_won', 'misc_aerialduels_lost', 'misc_aerialduels_won_perc', 'keeper_penaltykicks_pka',
                         'attendance']

object = StandardScaler()

df.loc[:,cols_to_transform] = object.fit_transform(df.loc[:,cols_to_transform])
df

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()


,result,gf,xg,captain,formation,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,...,keeper_penaltykicks_pka,schedule_time,attendance,referee,fbref_season,fbref_league_id,fbref_own,fbref_oppon,fbref_match_id,home
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-06,2,2.0,1.2,59,11,0.993065,-0.492178,0.678333,1.573056,1.379440,...,NaN,8,-0.364858,115,0,2,1,47,4713,1
2017-08-12,0,2.0,1.4,616,11,0.993065,0.202280,1.173772,1.189740,0.872919,...,-0.395817,27,-0.704719,120,0,2,1,93,4694,0
2017-08-20,2,1.0,0.6,19,11,-0.014119,-0.029206,0.182895,0.258828,0.062487,...,-0.395817,8,3.054934,119,0,2,1,91,4524,0
2017-08-27,2,1.0,0.8,19,11,-0.014119,-0.029206,-0.807981,-0.986950,0.062487,...,NaN,8,-0.491020,107,0,2,1,44,4714,1
2017-09-10,2,1.0,2.4,59,11,-0.014119,0.665252,0.678333,0.210914,-0.849250,...,-0.395817,2,1.164133,110,0,2,1,61,4582,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-08,0,1.0,1.0,175,12,-0.014119,-0.955151,0.678333,2.654556,0.569007,...,NaN,8,NaN,141,5,2,1,44,6602,1
2023-04-16,1,1.0,0.3,80,12,-0.014119,-0.955151,0.182895,1.675730,0.569007,...,4.799276,2,NaN,113,5,2,1,77,6450,0
2023-04-21,1,1.0,1.8,935,12,-0.014119,1.128224,0.182895,-0.521495,-0.241426,...,NaN,6,0.073005,126,5,2,1,102,6603,1


In [116]:
X = []
y = []
object = StandardScaler()
cols_to_transform = ['shooting_standard_gls', 'shooting_standard_sh', 'shooting_standard_sot', 
                         'shooting_standard_sot_perc', 'shooting_standard_g_per_sh', 'shooting_standard_g_per_sot',
                         'shooting_standard_dist', 'shooting_standard_fk', 'shooting_standard_pk', 
                         'shooting_standard_pkatt', 'shooting_expected_npxg', 'shooting_expected_npxg_per_sh',
                         'shooting_expected_g_minus_xg', 'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
                         'keeper_performance_saves', 'keeper_performance_save_perc', 'keeper_performance_cs',
                         'keeper_performance_psxg', 'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt', 
                         'keeper_penaltykicks_pksv', 'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
                         'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr', 'keeper_passes_launch_perc',
                         'keeper_passes_avglen', 'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc', 
                         'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp', 'keeper_crosses_stp_perc',
                         'keeper_sweeper_number_opa', 'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
                         'passing_total_cmp_perc', 'passing_total_totdist', 'passing_total_prgdist', 'passing_short_cmp',
                         'passing_short_att', 'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
                         'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att', 'passing_long_cmp_perc',
                         'passing_attacking_ast', 'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
                         'passing_attacking_1_per_3', 'passing_attacking_ppa', 'passing_attacking_crspa', 'passing_attacking_prgp',
                         'passing_types_passtypes_live', 'passing_types_passtypes_dead', 'passing_types_passtypes_fk',
                         'passing_types_passtypes_tb', 'passing_types_passtypes_sw', 'passing_types_passtypes_crs',
                         'passing_types_passtypes_ti', 'passing_types_passtypes_ck', 'passing_types_cornerkicks_in',
                         'passing_types_cornerkicks_out', 'passing_types_cornerkicks_str', 'passing_types_outcomes_off',
                         'passing_types_outcomes_blocks', 'gca_scatypes_sca', 'gca_scatypes_passlive', 'gca_scatypes_passdead',
                         'gca_scatypes_to', 'gca_scatypes_sh', 'gca_scatypes_fld', 'gca_scatypes_def', 'gca_gcatypes_gca', 
                         'gca_gcatypes_passlive', 'gca_gcatypes_passdead', 'gca_gcatypes_to', 'gca_gcatypes_sh',
                         'gca_gcatypes_fld',  'gca_gcatypes_def', 'defense_tackles_tkl', 'defense_tackles_tklw',
                         'defense_tackles_def3rd', 'defense_tackles_mid3rd', 'defense_tackles_att3rd', 'defense_challenges_tkl',
                         'defense_challenges_att', 'defense_challenges_tkl_perc', 'defense_challenges_lost', 'defense_blocks_blocks',
                         'defense_blocks_sh', 'defense_blocks_pass', 'defense_general_int', 'defense_general_tkl_plus_int',
                         'defense_general_clr', 'defense_general_err', 'possession_general_poss', 'possession_touches_touches',
                         'possession_touches_defpen', 'possession_touches_def3rd', 'possession_touches_mid3rd',
                         'possession_touches_att3rd', 'possession_touches_attpen', 'possession_touches_live', 'possession_takeons_att',
                         'possession_takeons_succ', 'possession_takeons_succ_perc', 'possession_takeons_tkld', 'possession_takeons_tkld_perc',
                         'possession_carries_carries', 'possession_carries_totdist', 'possession_carries_prgdist',
                         'possession_carries_prgc', 'possession_carries_1_per_3', 'possession_carries_cpa', 'possession_carries_mis',
                         'possession_carries_dis', 'possession_receiving_rec', 'possession_receiving_prgr',
                         'misc_performance_crdy', 'misc_performance_crdr', 'misc_performance_2crdy', 'misc_performance_fls',
                         'misc_performance_fld', 'misc_performance_off', 'misc_performance_og', 'misc_performance_recov',
                         'misc_aerialduels_won', 'misc_aerialduels_lost', 'misc_aerialduels_won_perc', 'keeper_penaltykicks_pka',
                         'attendance']


for team in set(list(new_data.data_frame.fbref_home_id.unique()) + list(new_data.data_frame.fbref_away_id.unique())):
    df = new_data.dataset_team(team)
    df.loc[:,cols_to_transform] = object.fit_transform(df.loc[:,cols_to_transform])
    for seas in df.fbref_season.unique():
        for i in range(len(df[df.fbref_season == seas])-1):
            #i = i + 1
            empty_df = np.zeros([38, df[df.fbref_season == seas].shape[1]])
            empty_df[-i - 1:,:] = df[df.fbref_season == seas].iloc[:i + 1]
            result = np.array(df[df.fbref_season == seas].iloc[i + 1].result)
            X.append(empty_df)
            y.append(result)

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_2420/3851513251.p

In [118]:
X1 = [torch.from_numpy(x.astype(float)) for x in X]
y1 = [torch.from_numpy(Y.astype(float)) for Y in y]
y1 = [torch.nn.functional.one_hot(x.long(), num_classes = 3) for x in y1]

In [120]:
X1[0]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 0.0000e+00, 5.0000e-01,  ..., 6.1000e+01, 4.5800e+03,
         0.0000e+00]], dtype=torch.float64)

In [121]:
class data_to_lstm():

    def __init__(self, mylist1, mylist2):
        self.output = []
        self.mylist1 = mylist1
        self.mylist2 = mylist2
        
        for i in range(len(mylist1)):
            self.output.append((mylist1[i], mylist2[i]))
        
 

    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.output[idx]


In [129]:
data_tansformed = data_to_lstm(X1,y1)
len_train = int(0.8 * len(X1))
len_test = len(X1) - len_train

train, test = torch.utils.data.random_split(data_tansformed, [len_train, len_test])
#test = torch.utils.data.random_split(data_tansformed, len_test)

In [140]:
train_loader = DataLoader(train, batch_size = 32, shuffle = True)
test_loader = DataLoader(test, batch_size = 32, shuffle = True)

In [213]:
class Sport_pred_LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features
        self.num_classes = num_classes
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers)

        self.l_linear = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.l_linear2 = torch.nn.Linear(self.n_hidden, num_classes)
        self.soft = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()
        
    #def init_hidden(self, batch_size):
     #   hidden_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
      #  cell_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
       # self.hidden = (hidden_state, cell_state)


    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        #print(x)
        h0 = torch.randn(self.n_layers*1, 38, self.n_hidden).to(x.dtype)
        #print(h0.dtype)
        c0 = torch.randn(self.n_layers*1, 38, self.n_hidden).to(x.dtype)
        #print(c0.dtype)
        
        lstm_out,(hn, cn) = self.l_lstm(x,(h0, c0))
        #print("lstm out:", lstm_out)
        #x = lstm_out.contiguous().view(-1, self.n_hidden)
        x = lstm_out[:, -1, :]
        #print(x)
        x = self.relu(self.l_linear(x))
        out = self.soft(self.l_linear2(x))
        #print(out)
        return out

In [214]:
model = Sport_pred_LSTM(152,38,3)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
epoch = 30
losses = list()
for epoch in range(epoch):
    model.eval()
    loss_test = 0
    accur = 0
    for i, (seq, target) in enumerate(test_loader):
        pred = model(seq)
        #print(pred.shape)
        #print(target.shape)
        loss_test += criterion(pred, target.to(torch.float32)).item()
        pred = torch.argmax(pred, dim = 1)
        target = torch.argmax(target, dim = 1)
        #losses += loss
        accur += pred.eq(target).sum().item()
    
    print(f" Loss: {loss} \n Accuracy {100 * accur / ((i + 1) * 32)}%")
    
    model.train()
    
    for i, (seq, target) in enumerate(train_loader):
        model.zero_grad()
        pred = model(seq)
        loss = criterion(pred, target.to(torch.float32))
        loss.backward()
        optimizer.step()

 Loss: 0.22065456211566925 
 Accuracy 26.5625%
 Loss: 0.22083884477615356 
 Accuracy 26.923076923076923%
 Loss: 0.222123920917511 
 Accuracy 28.75%
 Loss: 0.2279193252325058 
 Accuracy 31.634615384615383%
 Loss: 0.21734578907489777 
 Accuracy 33.70192307692308%
 Loss: 0.21990744769573212 
 Accuracy 36.17788461538461%
 Loss: 0.22095263004302979 
 Accuracy 36.17788461538461%
 Loss: 0.22415636479854584 
 Accuracy 36.68269230769231%
 Loss: 0.21830154955387115 
 Accuracy 36.85096153846154%
 Loss: 0.21481472253799438 
 Accuracy 36.58653846153846%
 Loss: 0.216144397854805 
 Accuracy 37.13942307692308%
 Loss: 0.22176431119441986 
 Accuracy 36.58653846153846%
 Loss: 0.21400441229343414 
 Accuracy 36.51442307692308%
 Loss: 0.21768547594547272 
 Accuracy 36.5625%
 Loss: 0.21896494925022125 
 Accuracy 36.49038461538461%
 Loss: 0.20585651695728302 
 Accuracy 36.5625%
 Loss: 0.21569214761257172 
 Accuracy 36.61057692307692%
 Loss: 0.2329302579164505 
 Accuracy 36.77884615384615%
 Loss: 0.20880711078

In [144]:
for _, (s,p) in enumerate(train_loader):
    print(s.shape)
    print(p.shape)
    break

torch.Size([32, 38, 152])
torch.Size([32, 3])


In [195]:
class Sport_pred_LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features
        self.num_classes = num_classes
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers)

        self.l_linear = torch.nn.Linear(self.n_hidden, num_classes)
        self.soft = torch.nn.Softmax(dim = 1)
        
    #def init_hidden(self, batch_size):
     #   hidden_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
      #  cell_state = torch.randn(self.n_layers,batch_size,self.n_hidden)
       # self.hidden = (hidden_state, cell_state)


    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        #print(x)
        h0 = torch.randn(self.n_layers*1, self.n_hidden).to(x.dtype)
        #print(h0.dtype)
        c0 = torch.randn(self.n_layers*1, self.n_hidden).to(x.dtype)
        #print(c0.dtype)
        
        lstm_out,_ = self.l_lstm(x,(h0, c0))
        #print("lstm out:", lstm_out)
        x = lstm_out.contiguous().view(-1, self.n_hidden)
        #print(x)
        out = self.soft(self.l_linear(x))
        #print(out)
        return out

In [196]:
n_features = len(X1[0]) # this is number of parallel inputs

# create NN
mv_net = Sport_pred_LSTM(n_features, 150, 3)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-4, weight_decay = 1e-3)

train_episodes = 500
batch_size = 30





In [197]:
mv_net.train()

for t in range(train_episodes):

    mv_net.train()
    for _, (data, target) in enumerate(dataloader):
        inpt = data.to(torch.float32)
        label = target
        #print(data.dtype)

        output = mv_net.forward(data) 
        print(output)
        loss = criterion(output, label.to(torch.float32)) 
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
        break
    if t%50 == 0:
        print('step : ' , t , 'loss : ' , loss.item())
        
    #mv_net.eval()
    

RuntimeError: For batched 3-D input, hx and cx should also be 3-D but got (2-D, 2-D) tensors

In [464]:
df.fbref_match_id

schedule_date
2017-08-19        0
2017-09-09        1
2017-09-20        2
2017-09-23        3
2017-10-14        4
              ...  
2023-03-15    10652
2023-03-18    10653
2023-04-08    10654
2023-04-15    10655
2023-04-27    10656
Name: fbref_match_id, Length: 10657, dtype: int64

In [280]:

def sequencizer(X, y, seq_len):
    x_t = []
    y_t = []
    for i in range(len(X)):
        if i + seq_len + 1 == len(X):
            break
        x_train = X[i:i + seq_len]
        y_train = y[i + seq_len + 1]
        
        x_t.append(x_train)
        y_t.append(y_train)
        
    return np.array(x_t), np.array(y_t)

    

In [472]:
new_data1

,schedule_time,schedule_round,schedule_day,result_home,gf_home,gf_away,xg_home,xg_away,attendance,captain_home,formation_home,referee,fbref_season,fbref_league_id,fbref_home_id,fbref_away_id,fbref_match_id,shooting_standard_gls_home,shooting_standard_sh_home,shooting_standard_sot_home,shooting_standard_sot_perc_home,shooting_standard_g_per_sh_home,shooting_standard_g_per_sot_home,shooting_standard_dist_home,shooting_standard_fk_home,shooting_standard_pk_home,shooting_standard_pkatt_home,shooting_expected_npxg_home,shooting_expected_npxg_per_sh_home,shooting_expected_g_minus_xg_home,shooting_expected_npg_minus_xg_home,keeper_performance_sota_home,keeper_performance_saves_home,keeper_performance_save_perc_home,keeper_performance_cs_home,keeper_performance_psxg_home,keeper_performance_psxg_plus_minus_home,keeper_penaltykicks_pkatt_home,keeper_penaltykicks_pka,keeper_penaltykicks_pksv_home,keeper_penaltykicks_pkm_home,keeper_launched_cmp_home,keeper_launched_att_home,keeper_launched_cmp_perc_home,keeper_passes_att_home,keeper_passes_thr_home,keeper_passes_launch_perc_home,keeper_passes_avglen_home,keeper_goalkicks_att_home,keeper_goalkicks_launch_perc_home,...,defense_tackles_tkl_away,defense_tackles_tklw_away,defense_tackles_def3rd_away,defense_tackles_mid3rd_away,defense_tackles_att3rd_away,defense_challenges_tkl_away,defense_challenges_att_away,defense_challenges_tkl_perc_away,defense_challenges_lost_away,defense_blocks_blocks_away,defense_blocks_sh_away,defense_blocks_pass_away,defense_general_int_away,defense_general_tkl_plus_int_away,defense_general_clr_away,defense_general_err_away,possession_general_poss_away,possession_touches_touches_away,possession_touches_defpen_away,possession_touches_def3rd_away,possession_touches_mid3rd_away,possession_touches_att3rd_away,possession_touches_attpen_away,possession_touches_live_away,possession_takeons_att_away,possession_takeons_succ_away,possession_takeons_succ_perc_away,possession_takeons_tkld_away,possession_takeons_tkld_perc_away,possession_carries_carries_away,possession_carries_totdist_away,possession_carries_prgdist_away,possession_carries_prgc_away,possession_carries_1_per_3_away,possession_carries_cpa_away,possession_carries_mis_away,possession_carries_dis_away,possession_receiving_rec_away,possession_receiving_prgr_away,misc_performance_crdy_away,misc_performance_crdr_away,misc_performance_2crdy_away,misc_performance_fls_away,misc_performance_fld_away,misc_performance_off_away,misc_performance_og_away,misc_performance_recov_away,misc_aerialduels_won_away,misc_aerialduels_lost_away,misc_aerialduels_won_perc_away
schedule_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-08-19,0,Matchweek 1,0,0,3.0,0.0,1.1,1.4,39226.0,998,0,0,0,0,132,76,0,3.0,17.0,7.0,41.2,0.18,0.43,19.8,2.0,0.0,0.0,1.1,0.07,1.9,1.9,2.0,1.0,100.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,5.0,8.0,62.5,29.0,6.0,24.1,27.4,4.0,25.0,...,17.0,15.0,7.0,9.0,1.0,3.0,19.0,15.8,16.0,13.0,5.0,8.0,9.0,26.0,23.0,0.0,38.0,538.0,50.0,176.0,265.0,104.0,12.0,537.0,12.0,10.0,83.3,2.0,16.7,257.0,1322.0,607.0,9.0,9.0,1.0,13.0,6.0,341.0,36.0,0.0,0.0,0.0,11.0,13.0,0.0,0.0,40.0,9.0,8.0,52.9
2017-09-09,0,Matchweek 3,0,0,3.0,0.0,1.3,0.6,39457.0,970,1,1,0,0,132,133,1,2.0,15.0,4.0,26.7,0.13,0.50,17.5,0.0,0.0,0.0,1.3,0.08,0.7,0.7,2.0,2.0,100.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0,10.0,16.0,62.5,27.0,2.0,29.6,33.7,13.0,61.5,...,18.0,10.0,8.0,8.0,2.0,4.0,15.0,26.7,11.0,10.0,4.0,6.0,15.0,33.0,22.0,0.0,42.0,521.0,60.0,148.0,250.0,126.0,13.0,521.0,2.0,1.0,50.0,1.0,50.0,264.0,1451.0,696.0,11.0,8.0,1.0,5.0,17.0,346.0,40.0,1.0,0.0,0.0,7.0,15.0,1.0,1.0,34.0,10.0,10.0,50.0
2017-09-20,1,Matchweek 5,1,0,1.0,0.0,0.9,0.4,35652.0,749,0,2,0,0,132,39,2,1.0,12.0,5.0,41.7,0.08,0.20,22.6,1.0,0.0,0.0,0.9,0.08,0.1,0.1,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,10.0,30.0,22.0,1.0,31.8,32.5,9.0,33.3,...,12.0,10.0,2.0,7.0,3.0,5.0,15.0,33.3,10.0,17.0,5.0,12.0,12.0,24.0,21.0,0.0,40.0,532.0,50.0,188.0,251.0,96.0,7.0,532.0,13.0,

In [473]:
empty_df = pd.DataFrame(columns = list(new_data.dataset_team(0)), index = [0])

for team in set(list(new_data1.fbref_home_id.unique()) + list(new_data1.fbref_away_id.unique())):
    empty_df = pd.concat([new_data.dataset_team(team), empty_df])

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.p

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/279171873.p

In [511]:
dts2.loc["2023-04-25"]

result                 1
gf                   0.0
xg                   0.8
captain              580
formation              1
                   ...  
fbref_league_id        4
fbref_own            121
fbref_oppon           99
fbref_match_id     10534
home                   0
Name: 2023-04-25, Length: 152, dtype: object

In [506]:
dts1#.loc["2022-08-13"]

,result,gf,xg,captain,formation,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,shooting_standard_g_per_sot,shooting_standard_dist,shooting_standard_fk,shooting_standard_pk,shooting_standard_pkatt,shooting_expected_npxg,shooting_expected_npxg_per_sh,shooting_expected_g_minus_xg,shooting_expected_npg_minus_xg,keeper_performance_sota,keeper_performance_saves,keeper_performance_save_perc,keeper_performance_cs,keeper_performance_psxg,keeper_performance_psxg_plus_minus,keeper_penaltykicks_pkatt,keeper_penaltykicks_pksv,keeper_penaltykicks_pkm,keeper_launched_cmp,keeper_launched_att,keeper_launched_cmp_perc,keeper_passes_att,keeper_passes_thr,keeper_passes_launch_perc,keeper_passes_avglen,keeper_goalkicks_att,keeper_goalkicks_launch_perc,keeper_goalkicks_avglen,keeper_crosses_opp,keeper_crosses_stp,keeper_crosses_stp_perc,keeper_sweeper_number_opa,keeper_sweeper_avgdist,passing_total_cmp,passing_total_att,passing_total_cmp_perc,passing_total_totdist,passing_total_prgdist,passing_short_cmp,passing_short_att,...,defense_blocks_sh,defense_blocks_pass,defense_general_int,defense_general_tkl_plus_int,defense_general_clr,defense_general_err,possession_general_poss,possession_touches_touches,possession_touches_defpen,possession_touches_def3rd,possession_touches_mid3rd,possession_touches_att3rd,possession_touches_attpen,possession_touches_live,possession_takeons_att,possession_takeons_succ,possession_takeons_succ_perc,possession_takeons_tkld,possession_takeons_tkld_perc,possession_carries_carries,possession_carries_totdist,possession_carries_prgdist,possession_carries_prgc,possession_carries_1_per_3,possession_carries_cpa,possession_carries_mis,possession_carries_dis,possession_receiving_rec,possession_receiving_prgr,misc_performance_crdy,misc_performance_crdr,misc_performance_2crdy,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_og,misc_performance_recov,misc_aerialduels_won,misc_aerialduels_lost,misc_aerialduels_won_perc,keeper_penaltykicks_pka,schedule_time,attendance,referee,fbref_season,fbref_league_id,fbref_own,fbref_oppon,fbref_match_id,home


In [512]:
df = new_data.data_frame
X = []
y = []
for team in set(list(empty_df.fbref_own.unique()) + list(empty_df.fbref_oppon.unique())):
    for seas in empty_df.fbref_season.unique():
        dts1 = empty_df[(empty_df.fbref_own == team) & (empty_df.fbref_season == seas)]
        for i in range(len(dts1)):
            i += 11
            date11 = dts1.index[i - 11]
            date12 = dts1.index[i - 1]
            
            dts2 = empty_df[(empty_df.fbref_own == dts1.iloc[i].fbref_oppon) & (empty_df.fbref_season == seas)]
            date21 = dts2.index[i - 11]
            date22 = dts2.index[i - 1]
            
            #if dts1.index[i] != dts2.index[i]:
             #   print(dts1.iloc[i])
              #  print(dts2.iloc[i])
            if dts1.iloc[i].fbref_match_id != dts2.iloc[i].fbref_match_id:
                print("Alarm")
            x = [np.array(dts1.loc[date11:date12]), np.array(dts2.loc[date21:date22])]
            X.append(x)
            y.append(dts1.iloc[i].result) 
            break
            if i == len(dts1)-1:
                break
    print(team)

0
Alarm
Alarm
1
Alarm
Alarm
2
3
4
5
6
7
8
9
10
11
Alarm
12
13
14
15
Alarm
16
17
18
Alarm
19
Alarm
20
Alarm
21
Alarm
22
23
24
25
Alarm
Alarm
26
Alarm
Alarm
Alarm
27
28
29
Alarm
30
31
32
33
34
Alarm
35
36
Alarm
37
Alarm
38
Alarm
39
40
41
42
Alarm
43
44
45
46
Alarm
47
Alarm
48
49
Alarm
50
Alarm
51
52
53
Alarm
Alarm
54
Alarm
Alarm
Alarm
55
56
57
58
59
Alarm
60
61
Alarm
62
63
Alarm
64
65
66
Alarm
Alarm
Alarm
67
68
Alarm
Alarm
69
70
Alarm
71
72
Alarm
Alarm
73
74
Alarm
75
76
77
78
79
80
81
Alarm
Alarm
82
83
Alarm
84
85
86
Alarm
87
88
89
Alarm
90
Alarm
91
92
Alarm
93
94
95
96
Alarm
97
98
Alarm
99
100
101
102
103
104
Alarm
105
Alarm
Alarm
106
Alarm
107
Alarm
108
Alarm
109
110
111
Alarm
Alarm
112
Alarm
Alarm
113
Alarm
114
Alarm
Alarm
115
116
117
118
Alarm
119
120
Alarm
121
122
Alarm
123
Alarm
124
125
Alarm
Alarm
Alarm
126
127
128
129
Alarm
130
131
Alarm
132
133
Alarm
134
135
136
137
Alarm
138
139
Alarm
140
nan


In [481]:
empty_df[(empty_df.fbref_own == team) & (empty_df.fbref_season == seas)].index[10]

'2017-11-04'

In [462]:
X = [torch.from_numpy(x.astype(float)) for z in X for x in z]
y = [torch.from_numpy(Y.astype(float)) for Y in y]
y = [torch.nn.functional.one_hot(x.long(), num_classes = 3) for x in y]

ValueError: could not convert string to float: '7c834541'

In [448]:
class data_to_2lstm():

    def __init__(self, mylist1, mylist2):
        self.output =[]
        
        for i in range(len(mylist1)):
            self.output.append((mylist1[i][0], mylist1[i][1], mylist2[i]))
        
 

    def __len__(self):
        return len(self.output1)

    def __getitem__(self, idx):
        return self.output[idx]


In [449]:
dt2lstm = data_to_2lstm(X,y)
dataloader = DataLoader(dt2lstm, batch_size = 30,
                        shuffle = False)
dataloader

In [452]:
dt2lstm.__getitem__(0)

(array([[1, 0.0, 0.4, ..., 140, '7c834541', 0],
        [0, 1.0, 0.7, ..., 82, '37f97732', 1],
        [2, 1.0, 1.3, ..., 105, 'abccb71c', 0],
        ...,
        [1, 2.0, 0.5, ..., 112, 'f81f42ed', 0],
        [1, 1.0, 1.2, ..., 77, '7da1fc42', 1],
        [0, 1.0, 1.1, ..., 135, '7834b95f', 0]], dtype=object),
 array([[0, 1.0, 0.6, ..., 0, '7c834541', 1],
        [2, 1.0, 0.6, ..., 112, '424dc3f9', 0],
        [1, 0.0, 0.4, ..., 79, '5a276f0b', 0],
        ...,
        [2, 1.0, 1.5, ..., 46, '1585521f', 0],
        [1, 2.0, 1.0, ..., 82, '73da64ff', 1],
        [1, 0.0, 0.6, ..., 13, 'f6cf8af5', 0]], dtype=object),
 0)

In [440]:
for i, (x1, x2, y) in enumerate(dataloader):
    print(i, x1, y)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [388]:

X = {}
y = []

#set(list(df.fbref_away_id.unique()) + list(df.fbref_home_id.unique()))
for team in set(list(df.fbref_away_id.unique()) + list(df.fbref_home_id.unique())):
    data_team = new_data.dataset_team(team)
    for seas in data_team.fbref_season.unique():
        dts = data_team[data_team.fbref_season == seas]
        for i in range(len(dts)):
            string = str(team) + "_" + str(seas) + "_"
            X.append(dts.iloc[i:i + 10])
            y.append(dts.iloc[i + 10 + 1].result)
            #print(dts.iloc[i:i+10].result)
            if i + 10 + 1 == len(dts)-1:
                break



/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/359767110

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/359767110

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/359767110

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/359767110

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/359767110

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/3597671105.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True).copy()
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_28996/359767110

In [6]:
X[0].shape

NameError: name 'X' is not defined

In [368]:
#dts.iloc[0:4]
X = []
y = []
for i in range(len(dts)):
    X.append(dts.iloc[i:i + 10])
    y.append(dts.iloc[i + 10 + 1])
    #print(dts.iloc[i:i+10].result)
    if i + 10 + 1 == len(dts)-1:
        break

In [465]:
empty_df

,result,gf,xg,captain,formation,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,shooting_standard_g_per_sot,shooting_standard_dist,shooting_standard_fk,shooting_standard_pk,shooting_standard_pkatt,shooting_expected_npxg,shooting_expected_npxg_per_sh,shooting_expected_g_minus_xg,shooting_expected_npg_minus_xg,keeper_performance_sota,keeper_performance_saves,keeper_performance_save_perc,keeper_performance_cs,keeper_performance_psxg,keeper_performance_psxg_plus_minus,keeper_penaltykicks_pkatt,keeper_penaltykicks_pksv,keeper_penaltykicks_pkm,keeper_launched_cmp,keeper_launched_att,keeper_launched_cmp_perc,keeper_passes_att,keeper_passes_thr,keeper_passes_launch_perc,keeper_passes_avglen,keeper_goalkicks_att,keeper_goalkicks_launch_perc,keeper_goalkicks_avglen,keeper_crosses_opp,keeper_crosses_stp,keeper_crosses_stp_perc,keeper_sweeper_number_opa,keeper_sweeper_avgdist,passing_total_cmp,passing_total_att,passing_total_cmp_perc,passing_total_totdist,passing_total_prgdist,passing_short_cmp,passing_short_att,...,defense_blocks_sh,defense_blocks_pass,defense_general_int,defense_general_tkl_plus_int,defense_general_clr,defense_general_err,possession_general_poss,possession_touches_touches,possession_touches_defpen,possession_touches_def3rd,possession_touches_mid3rd,possession_touches_att3rd,possession_touches_attpen,possession_touches_live,possession_takeons_att,possession_takeons_succ,possession_takeons_succ_perc,possession_takeons_tkld,possession_takeons_tkld_perc,possession_carries_carries,possession_carries_totdist,possession_carries_prgdist,possession_carries_prgc,possession_carries_1_per_3,possession_carries_cpa,possession_carries_mis,possession_carries_dis,possession_receiving_rec,possession_receiving_prgr,misc_performance_crdy,misc_performance_crdr,misc_performance_2crdy,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_og,misc_performance_recov,misc_aerialduels_won,misc_aerialduels_lost,misc_aerialduels_won_perc,keeper_penaltykicks_pka,schedule_time,attendance,referee,fbref_season,fbref_league_id,fbref_own,fbref_oppon,fbref_match_id,home
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-12,1,0.0,0.4,556,3,0.0,9.0,1.0,11.1,0.00,0.00,22.6,0.0,NaN,0.0,0.4,0.04,-0.4,-0.4,4.0,3.0,75.0,0.0,0.6,-0.4,0.0,0.0,0.0,10.0,18.0,55.6,21.0,4.0,57.1,39.5,8.0,75.0,55.1,13.0,0.0,0.0,1.0,14.5,233.0,356.0,65.4,4563.0,2068.0,97.0,114.0,...,2.0,8.0,24.0,42.0,26.0,1.0,40.0,473.0,49.0,154.0,192.0,135.0,16.0,473.0,13.0,10.0,76.9,3.0,23.1,190.0,1434.0,729.0,12.0,10.0,5.0,17.0,14.0,227.0,33.0,1.0,0.0,0.0,10.0,13.0,6.0,0.0,38.0,23.0,19.0,54.8,0.0,2,39045.0,177,0,4,0,140,7c834541,0
2017-08-19,0,1.0,0.7,556,3,1.0,11.0,4.0,36.4,0.09,0.25,15.4,0.0,0.0,0.0,0.7,0.06,0.3,0.3,6.0,6.0,100.0,1.0,1.1,1.1,0.0,0.0,0.0,15.0,37.0,40.5,30.0,3.0,83.3,57.1,12.0,100.0,64.1,20.0,1.0,5.0,2.0,11.4,151.0,243.0,62.1,3390.0,1900.0,63.0,80.0,...,6.0,8.0,8.0,27.0,37.0,1.0,24.0,356.0,85.0,168.0,132.0,66.0,10.0,356.0,12.0,7.0,58.3,5.0,41.7,141.0,1117.0,524.0,8.0,8.0,3.0,10.0,10.0,146.0,16.0,0.0,0.0,0.0,6.0,11.0,4.0,0.0,38.0,17.0,9.0,65.4,NaN,34,29459.0,186,0,4,0,82,37f97732,1
2017-08-27,2,1.0,1.3,556,3,1.0,16.0,2.0,12.5,0.06,0.50,19.0,0.0,NaN,0.0,1.3,0.08,-0.3,-0.3,2.0,1.0,50.0,0.0,0.8,-0.2,0.0,0.0,0.0,1.0,13.0,7.7,24.0,6.0,37.5,34.5,7.0,57.1,47.6,25.0,3.0,12.0,0.0,9.0,406.0,485.0,83.7,7409.0,2562.0,190.0,209.0,...,0.0,5.0,7.0,27.0,45.0,0.0,57.0,616.0,57.0,162.0,282.0,178.0,23.0,616.0,19.0,10.0,52.6,9.0,47.4,322.0,1876.0,1102.0,21.0,15.0,3.0,18.0,8.0,403.0,54.0,2.0,0.0,0.0,9.0,10.0,0.0,0.0,26.0,28.0,12.0,70.0,0.0,13,22704.0,188,

In [377]:
np.array(X[0])
#X[0]

array([[1, 0.0, 0.4, ..., 140, '7c834541', 0],
       [0, 1.0, 0.7, ..., 82, '37f97732', 1],
       [2, 1.0, 1.3, ..., 105, 'abccb71c', 0],
       ...,
       [1, 2.0, 0.5, ..., 112, 'f81f42ed', 0],
       [1, 1.0, 1.2, ..., 77, '7da1fc42', 1],
       [0, 1.0, 1.1, ..., 135, '7834b95f', 0]], dtype=object)

In [284]:
class Sport_pred_2LSTM(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = 2 # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers)
        
        self.l_lstm2 = torch.nn.LSTM(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers)

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(self.n_hidden, self.n_hidden * 0.5)
        self.l_linear2 = torch.nn.Linear(self.n_hidden * 0.5, self.n_hidden * 0.5)
        self.l_linear3 = torch.nn.Linear(self.n_hidden * 0.5, num_classes)
        
        self.soft = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        
        # initialize values for two lstms
        h01 = torch.randn(self.n_layers*1, self.n_hidden).to(x.dtype)
        h02 = torch.randn(self.n_layers*1, self.n_hidden).to(y.dtype)
        c01 = torch.randn(self.n_layers*1, self.n_hidden).to(x.dtype)
        c02 = torch.randn(self.n_layers*1, self.n_hidden).to(y.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,_ = self.l_lstm1(x,(h01, c01))
        lstm_out2,_ = self.l_lstm2(x,(h02, c02))
        x = lstm_out1.contiguous().view(-1, self.n_hidden)
        y = lstm_out1.contiguous().view(-1, self.n_hidden)
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        out_lay1 = self.l_linear1(in_lay1)
        in_lay2 = self.relu(out_lay1)
        out_lay2 = self.l_linear2(in_lay2)
        in_lay3 = self.relu(out_lay3)
        out = self.soft(self.l_linear3(in_lay3))
        
        #print(out)
        return out

In [285]:
class Sport_pred_standard_nn(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes):
        super(Sport_pred_LSTM, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(self.n_features * 2, self.n_hidden * 2)
        self.l_linear2 = torch.nn.Linear(self.n_hidden * 2, self.n_hidden * 0.5)
        self.l_linear3 = torch.nn.Linear(self.n_hidden * 0.5, num_classes)
        
        self.soft = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()


    def forward(self, x, y):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        y = y.to(torch.float32)
        y = torch.nan_to_num(y, nan = 0.0)    
        
        # run lstm output through nn to predict outcome
        in_lay1 = torch.cat((x, y), 1)
        out_lay1 = self.l_linear1(in_lay1)
        in_lay2 = self.relu(out_lay1)
        out_lay2 = self.l_linear2(in_lay2)
        in_lay3 = self.relu(out_lay3)
        out = self.soft(self.l_linear3(in_lay3))

        return out

In [290]:
class PositionalEncoder(torch.nn.Module):

    def __init__(self, dropout: float = 0.1, max_seq_len: int = 5000, d_model: int = 512, batch_first: bool = False):

        """
        Parameters:
            dropout: the dropout rate
            max_seq_len: the maximum length of the input sequences
            d_model: The dimension of the output of sub-layers in the model 
                     (Vaswani et al, 2017)
        """

        super().__init__()

        self.d_model = d_model
        
        self.dropout = nn.Dropout(p=dropout)

        self.batch_first = batch_first

        self.x_dim = 1 if batch_first else 0

        # copy pasted from PyTorch tutorial
        position = torch.arange(max_seq_len).unsqueeze(1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        
        pe = torch.zeros(max_seq_len, 1, d_model)
        
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, enc_seq_len, dim_val] or 
               [enc_seq_len, batch_size, dim_val]
        """

        x = x + self.pe[:x.size(self.x_dim)]

        return self.dropout(x)


In [291]:
class Sport_pred_Transformer(torch.nn.Module):

    def __init__(self, input_size: int, dec_seq_len: int, batch_first: bool, out_seq_len: int = 58, 
                 dim_val: int = 512, n_encoder_layers: int = 4, n_decoder_layers: int = 4, n_heads: int = 8,
                 dropout_encoder: float = 0.2, dropout_decoder: float = 0.2, dropout_pos_enc: float = 0.1,
                 dim_feedforward_encoder: int = 2048, dim_feedforward_decoder: int = 2048, 
                 num_predicted_features: int = 1): 

        """
        Args:
            input_size: int, number of input variables. 1 if univariate.
            dec_seq_len: int, the length of the input sequence fed to the decoder
            dim_val: int, aka d_model. All sub-layers in the model produce 
                     outputs of dimension dim_val
            n_encoder_layers: int, number of stacked encoder layers in the encoder
            n_decoder_layers: int, number of stacked encoder layers in the decoder
            n_heads: int, the number of attention heads (aka parallel attention layers)
            dropout_encoder: float, the dropout rate of the encoder
            dropout_decoder: float, the dropout rate of the decoder
            dropout_pos_enc: float, the dropout rate of the positional encoder
            dim_feedforward_encoder: int, number of neurons in the linear layer 
                                     of the encoder
            dim_feedforward_decoder: int, number of neurons in the linear layer 
                                     of the decoder
            num_predicted_features: int, the number of features you want to predict.
                                    Most of the time, this will be 1 because we're
                                    only forecasting FCR-N prices in DK2, but in
                                    we wanted to also predict FCR-D with the same
                                    model, num_predicted_features should be 2.
        """

        super().__init__() 

        self.dec_seq_len = dec_seq_len

        #print("input_size is: {}".format(input_size))
        #print("dim_val is: {}".format(dim_val))

        # Creating the three linear layers needed for the model
        self.encoder_input_layer = nn.Linear(
            in_features = input_size, 
            out_features = dim_val 
            )

        self.decoder_input_layer = nn.Linear(
            in_features = num_predicted_features,
            out_features = dim_val
            )  
        
        self.linear_mapping = nn.Linear(
            in_features = dim_val, 
            out_features = num_predicted_features
            )

        # Create positional encoder
        self.positional_encoding_layer = pe.PositionalEncoder(
            d_model = dim_val,
            dropout = dropout_pos_enc
            )

        # The encoder layer used in the paper is identical to the one used by
        # Vaswani et al (2017) on which the PyTorch module is based.
        encoder_layer = nn.TransformerEncoderLayer(
            d_model = dim_val, 
            nhead = n_heads,
            dim_feedforward = dim_feedforward_encoder,
            dropout = dropout_encoder,
            batch_first = batch_first
            )

        # Stack the encoder layers in nn.TransformerDecoder
        # It seems the option of passing a normalization instance is redundant
        # in my case, because nn.TransformerEncoderLayer per default normalizes
        # after each sub-layer
        # (https://github.com/pytorch/pytorch/issues/24930).
        self.encoder = nn.TransformerEncoder(
            encoder_layer = encoder_layer,
            num_layers = n_encoder_layers, 
            norm = None
            )

        decoder_layer = nn.TransformerDecoderLayer(
            d_model = dim_val,
            nhead = n_heads,
            dim_feedforward = dim_feedforward_decoder,
            dropout = dropout_decoder,
            batch_first = batch_first
            )

        # Stack the decoder layers in nn.TransformerDecoder
        # It seems the option of passing a normalization instance is redundant
        # in my case, because nn.TransformerDecoderLayer per default normalizes
        # after each sub-layer
        # (https://github.com/pytorch/pytorch/issues/24930).
        self.decoder = nn.TransformerDecoder(
            decoder_layer = decoder_layer,
            num_layers = n_decoder_layers, 
            norm = None
            )

    def forward(self, src: torch.Tensor, tgt: torch.Tensor, src_mask: torch.Tensor = None, 
                tgt_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Returns a tensor of shape:
        [target_sequence_length, batch_size, num_predicted_features]
        
        Args:
            src: the encoder's output sequence. Shape: (S,E) for unbatched input, 
                 (S, N, E) if batch_first=False or (N, S, E) if 
                 batch_first=True, where S is the source sequence length, 
                 N is the batch size, and E is the number of features (1 if univariate)
            tgt: the sequence to the decoder. Shape: (T,E) for unbatched input, 
                 (T, N, E)(T,N,E) if batch_first=False or (N, T, E) if 
                 batch_first=True, where T is the target sequence length, 
                 N is the batch size, and E is the number of features (1 if univariate)
            src_mask: the mask for the src sequence to prevent the model from 
                      using data points from the target sequence
            tgt_mask: the mask for the tgt sequence to prevent the model from
                      using data points from the target sequence
        """

        #print("From model.forward(): Size of src as given to forward(): {}".format(src.size()))
        #print("From model.forward(): tgt size = {}".format(tgt.size()))

        # Pass throguh the input layer right before the encoder
        src = self.encoder_input_layer(src) # src shape: [batch_size, src length, dim_val] regardless of number of input features
        #print("From model.forward(): Size of src after input layer: {}".format(src.size()))

        # Pass through the positional encoding layer
        src = self.positional_encoding_layer(src) # src shape: [batch_size, src length, dim_val] regardless of number of input features
        #print("From model.forward(): Size of src after pos_enc layer: {}".format(src.size()))

        # Pass through all the stacked encoder layers in the encoder
        # Masking is only needed in the encoder if input sequences are padded
        # which they are not in this time series use case, because all my
        # input sequences are naturally of the same length. 
        # (https://github.com/huggingface/transformers/issues/4083)
        src = self.encoder( # src shape: [batch_size, enc_seq_len, dim_val]
            src=src
            )
        #print("From model.forward(): Size of src after encoder: {}".format(src.size()))

        # Pass decoder input through decoder input layer
        decoder_output = self.decoder_input_layer(tgt) # src shape: [target sequence length, batch_size, dim_val] regardless of number of input features
        #print("From model.forward(): Size of decoder_output after linear decoder layer: {}".format(decoder_output.size()))

        #if src_mask is not None:
            #print("From model.forward(): Size of src_mask: {}".format(src_mask.size()))
        #if tgt_mask is not None:
            #print("From model.forward(): Size of tgt_mask: {}".format(tgt_mask.size()))

        # Pass throguh decoder - output shape: [batch_size, target seq len, dim_val]
        decoder_output = self.decoder(
            tgt=decoder_output,
            memory=src,
            tgt_mask=tgt_mask,
            memory_mask=src_mask
            )

        #print("From model.forward(): decoder_output shape after decoder: {}".format(decoder_output.shape))

        # Pass through linear mapping
        decoder_output = self.linear_mapping(decoder_output) # shape [batch_size, target seq len]
        #print("From model.forward(): decoder_output size after linear_mapping = {}".format(decoder_output.size()))

        return decoder_output


In [292]:
## Model parameters
dim_val = 512 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_decoder_layers = 4 # Number of times the decoder layer is stacked in the decoder
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 1 # The number of input variables. 1 if univariate forecasting.
dec_seq_len = 92 # length of input given to decoder. Can have any integer value.
enc_seq_len = 153 # length of input given to encoder. Can have any integer value.
output_sequence_length = 58 # Length of the target sequence, i.e. how many time steps should your forecast cover
max_seq_len = enc_seq_len # What's the longest sequence the model will encounter? Used to make the positional encoder

model = tst.TimeSeriesTransformer(
    dim_val=dim_val,
    input_size=input_size, 
    dec_seq_len=dec_seq_len,
    max_seq_len=max_seq_len,
    out_seq_len=output_sequence_length, 
    n_decoder_layers=n_decoder_layers,
    n_encoder_layers=n_encoder_layers,
    n_heads=n_heads)


NameError: name 'tst' is not defined

In [293]:

def get_src_trg(
        self,
        sequence: torch.Tensor, 
        enc_seq_len: int, 
        target_seq_len: int
        ) -> Tuple[torch.tensor, torch.tensor, torch.tensor]:

        """
        Generate the src (encoder input), trg (decoder input) and trg_y (the target)
        sequences from a sequence. 
        Args:
            sequence: tensor, a 1D tensor of length n where 
                    n = encoder input length + target sequence length  
            enc_seq_len: int, the desired length of the input to the transformer encoder
            target_seq_len: int, the desired length of the target sequence (the 
                            one against which the model output is compared)
        Return: 
            src: tensor, 1D, used as input to the transformer model
            trg: tensor, 1D, used as input to the transformer model
            trg_y: tensor, 1D, the target sequence against which the model output
                is compared when computing loss. 
        
        """
        #print("Called dataset.TransformerDataset.get_src_trg")
        assert len(sequence) == enc_seq_len + target_seq_len, "Sequence length does not equal (input length + target length)"
        
        #print("From data.TransformerDataset.get_src_trg: sequence shape: {}".format(sequence.shape))

        # encoder input
        src = sequence[:enc_seq_len] 
        
        # decoder input. As per the paper, it must have the same dimension as the 
        # target sequence, and it must contain the last value of src, and all
        # values of trg_y except the last (i.e. it must be shifted right by 1)
        trg = sequence[enc_seq_len-1:len(sequence)-1]

        #print("From data.TransformerDataset.get_src_trg: trg shape before slice: {}".format(trg.shape))

        trg = trg[:, 0]

        #print("From data.TransformerDataset.get_src_trg: trg shape after slice: {}".format(trg.shape))

        if len(trg.shape) == 1:

            trg = trg.unsqueeze(-1)

            #print("From data.TransformerDataset.get_src_trg: trg shape after unsqueeze: {}".format(trg.shape))

        
        assert len(trg) == target_seq_len, "Length of trg does not match target sequence length"

        # The target sequence against which the model output will be compared to compute loss
        trg_y = sequence[-target_seq_len:]

        #print("From data.TransformerDataset.get_src_trg: trg_y shape before slice: {}".format(trg_y.shape))

        # We only want trg_y to consist of the target variable not any potential exogenous variables
        trg_y = trg_y[:, 0]

        #print("From data.TransformerDataset.get_src_trg: trg_y shape after slice: {}".format(trg_y.shape))

        assert len(trg_y) == target_seq_len, "Length of trg_y does not match target sequence length"

        return src, trg, trg_y.squeeze(-1) # change size from [batch_size, target_seq_len, num_features] to [batch_size, target_seq_len] 


NameError: name 'Tuple' is not defined

In [ ]:

def generate_square_subsequent_mask(dim1: int, dim2: int) -> Tensor:
    """
    Generates an upper-triangular matrix of -inf, with zeros on diag.
    Source:
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    Args:
        dim1: int, for both src and tgt masking, this must be target sequence
              length
        dim2: int, for src masking this must be encoder sequence length (i.e. 
              the length of the input sequence to the model), 
              and for tgt masking, this must be target sequence length 
    Return:
        A Tensor of shape [dim1, dim2]
    """
    return torch.triu(torch.ones(dim1, dim2) * float('-inf'), diagonal=1)

# Input length
enc_seq_len = 100

# Output length
output_sequence_length = 58

# Make src mask for decoder with size:
tgt_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=output_sequence_length
   )

src_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=enc_seq_len
    )


In [ ]:

output = model(
    src=src, 
    tgt=trg,
    src_mask=src_mask,
    tgt_mask=tgt_mask
    )

In [86]:
for input_,target in dataloader:
    print(input_.shape)
    net = torch.nn.LSTM(288,10,1)
    h0 = torch.randn(1, 10)
    c0 = torch.randn(1, 10)
    output, (hn, cn) = net(input_, (h0, c0))
    break

torch.Size([30, 288])


RuntimeError: mat1 and mat2 must have the same dtype

In [95]:
data.to(torch.float32)

tensor([[ 1.0470e+03,  0.0000e+00, -7.5423e-01,  ...,  0.0000e+00,
          1.2000e+01,  0.0000e+00],
        [ 1.0470e+03,  0.0000e+00, -1.5763e+00,  ...,  0.0000e+00,
          1.2000e+01,  0.0000e+00],
        [ 1.0470e+03,  0.0000e+00, -1.9825e-02,  ...,  0.0000e+00,
          1.2000e+01,  0.0000e+00],
        ...,
        [ 1.0060e+03,  0.0000e+00, -1.2058e+00,  ...,  1.0000e+00,
          1.2000e+01,  0.0000e+00],
        [ 1.0060e+03,  0.0000e+00, -4.2011e-01,  ...,  2.0000e+00,
          1.2000e+01,  0.0000e+00],
        [ 1.0060e+03,  0.0000e+00, -2.2493e-01,  ...,  2.0000e+00,
          1.2000e+01,  0.0000e+00]])

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) # fully connected 
        self.fc_2 = nn.Linear(128, num_classes) # fully connected last layer
        self.relu = nn.ReLU()
        
    def forward(self,x):
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        return out

In [299]:
def data_to_input(data_frame):
    club_np = data_frame.to_numpy()
    X = []
    y = []
    for cl in range(window_size, len(club_np)-1):
        x = [[a] for a in club_np[cl-window_size:cl,3:]]
        label = club_np[cl+1,0]
        X.append(x)
        y.append(label)
    return X,y


In [295]:

 

object= StandardScaler()
 
# Splitting the independent and dependent variables
#i_data = dataset.data
#response = dataset.target
 
# standardization 
scale = object.fit_transform(club_np[:,10:12]) 
print(scale)

NameError: name 'club_np' is not defined

In [297]:
scale = object.fit_transform(d)
print(scale)

[[        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 ...
 [-0.74265998  0.36241024  2.2731764  ...  0.         -0.2657936
  -2.14103381]
 [-1.52490839  3.6499926   3.16798574 ...  0.         -0.26521127
   0.12707839]
 [-1.91495511  1.72677782  2.61776863 ...  0.         -0.00576638
   1.25879623]]


/Users/paul/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:980: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/paul/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/Users/paul/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1005: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [300]:
scale[10].shape

(150,)

In [362]:
a = club.fbref_own.unique() + club.fbref_oppon.unique()
for i in set(a):
    print(i)

e0652b021d8099f8
e0652b02c4260e09
e0652b02ffcbe334
e0652b021d2fe027
e0652b02d609edc0
e0652b02421387cf
e0652b02cc919b35
e0652b02dc56fe14
e0652b02105360fe
e0652b02cf74a709
e0652b02922493f3
e0652b024ef57aeb
e0652b028ff9e3b3
e0652b0204eea015
e0652b02c5577084
e0652b026a7ad59d
e0652b024fcb34fd
e0652b02eab4234c
e0652b029aad3a77
e0652b0268449f6d
e0652b02658bf2de
e0652b02a3d88bd8
e0652b027213da33
e0652b0221680aa4
e0652b020e72edf2
e0652b023074d7b1
e0652b02d48ad4ff
e0652b02af5d5982
e0652b02e2befd26


In [317]:
list(club.columns)

['result',
 'gf',
 'xg',
 'captain',
 'formation',
 'shooting_standard_gls',
 'shooting_standard_sh',
 'shooting_standard_sot',
 'shooting_standard_sot_perc',
 'shooting_standard_g_per_sh',
 'shooting_standard_g_per_sot',
 'shooting_standard_dist',
 'shooting_standard_fk',
 'shooting_standard_pk',
 'shooting_standard_pkatt',
 'shooting_expected_npxg',
 'shooting_expected_npxg_per_sh',
 'shooting_expected_g_minus_xg',
 'shooting_expected_npg_minus_xg',
 'keeper_performance_sota',
 'keeper_performance_saves',
 'keeper_performance_save_perc',
 'keeper_performance_cs',
 'keeper_performance_psxg',
 'keeper_performance_psxg_plus_minus',
 'keeper_penaltykicks_pkatt',
 'keeper_penaltykicks_pksv',
 'keeper_penaltykicks_pkm',
 'keeper_launched_cmp',
 'keeper_launched_att',
 'keeper_launched_cmp_perc',
 'keeper_passes_att',
 'keeper_passes_thr',
 'keeper_passes_launch_perc',
 'keeper_passes_avglen',
 'keeper_goalkicks_att',
 'keeper_goalkicks_launch_perc',
 'keeper_goalkicks_avglen',
 'keeper_cro

In [178]:
features = ['shooting_standard_gls',
 'shooting_standard_sh',
 'shooting_standard_sot',
 'shooting_standard_sot_perc',
 'shooting_standard_g_per_sh',
 'shooting_standard_g_per_sot',
 'shooting_standard_dist',
 'shooting_standard_fk',
 'shooting_standard_pk',
 'shooting_standard_pkatt',
 'shooting_expected_npxg',
 'shooting_expected_npxg_per_sh',
 'shooting_expected_g_minus_xg',
 'shooting_expected_npg_minus_xg',
 'keeper_performance_sota',
 'keeper_performance_saves',
 'keeper_performance_save_perc',
 'keeper_performance_cs',
 'keeper_performance_psxg',
 'keeper_performance_psxg_plus_minus',
 'keeper_penaltykicks_pkatt',
 'keeper_penaltykicks_pksv',
 'keeper_penaltykicks_pkm',
 'keeper_launched_cmp',
 'keeper_launched_att',
 'keeper_launched_cmp_perc',
 'keeper_passes_att',
 'keeper_passes_thr',
 'keeper_passes_launch_perc',
 'keeper_passes_avglen',
 'keeper_goalkicks_att',
 'keeper_goalkicks_launch_perc',
 'keeper_goalkicks_avglen',
 'keeper_crosses_opp',
 'keeper_crosses_stp',
 'keeper_crosses_stp_perc',
 'keeper_sweeper_number_opa',
 'keeper_sweeper_avgdist',
 'passing_total_cmp',
 'passing_total_att',
 'passing_total_cmp_perc',
 'passing_total_totdist',
 'passing_total_prgdist',
 'passing_short_cmp',
 'passing_short_att',
 'passing_short_cmp_perc',
 'passing_medium_cmp',
 'passing_medium_att',
 'passing_medium_cmp_perc',
 'passing_long_cmp',
 'passing_long_att',
 'passing_long_cmp_perc',
 'passing_attacking_ast',
 'passing_attacking_xag',
 'passing_attacking_xa',
 'passing_attacking_kp',
 'passing_attacking_1_per_3',
 'passing_attacking_ppa',
 'passing_attacking_crspa',
 'passing_attacking_prgp',
 'passing_types_passtypes_live',
 'passing_types_passtypes_dead',
 'passing_types_passtypes_fk',
 'passing_types_passtypes_tb',
 'passing_types_passtypes_sw',
 'passing_types_passtypes_crs',
 'passing_types_passtypes_ti',
 'passing_types_passtypes_ck',
 'passing_types_cornerkicks_in',
 'passing_types_cornerkicks_out',
 'passing_types_cornerkicks_str',
 'passing_types_outcomes_off',
 'passing_types_outcomes_blocks',
 'gca_scatypes_sca',
 'gca_scatypes_passlive',
 'gca_scatypes_passdead',
 'gca_scatypes_to',
 'gca_scatypes_sh',
 'gca_scatypes_fld',
 'gca_scatypes_def',
 'gca_gcatypes_gca',
 'gca_gcatypes_passlive',
 'gca_gcatypes_passdead',
 'gca_gcatypes_to',
 'gca_gcatypes_sh',
 'gca_gcatypes_fld',
 'gca_gcatypes_def',
 'defense_tackles_tkl',
 'defense_tackles_tklw',
 'defense_tackles_def3rd',
 'defense_tackles_mid3rd',
 'defense_tackles_att3rd',
 'defense_challenges_tkl',
 'defense_challenges_att',
 'defense_challenges_tkl_perc',
 'defense_challenges_lost',
 'defense_blocks_blocks',
 'defense_blocks_sh',
 'defense_blocks_pass',
 'defense_general_int',
 'defense_general_tkl_plus_int',
 'defense_general_clr',
 'defense_general_err',
 'possession_general_poss',
 'possession_touches_touches',
 'possession_touches_defpen',
 'possession_touches_def3rd',
 'possession_touches_mid3rd',
 'possession_touches_att3rd',
 'possession_touches_attpen',
 'possession_touches_live',
 'possession_takeons_att',
 'possession_takeons_succ',
 'possession_takeons_succ_perc',
 'possession_takeons_tkld',
 'possession_takeons_tkld_perc',
 'possession_carries_carries',
 'possession_carries_totdist',
 'possession_carries_prgdist',
 'possession_carries_prgc',
 'possession_carries_1_per_3',
 'possession_carries_cpa',
 'possession_carries_mis',
 'possession_carries_dis',
 'possession_receiving_rec',
 'possession_receiving_prgr',
 'misc_performance_crdy',
 'misc_performance_crdr',
 'misc_performance_2crdy',
 'misc_performance_fls',
 'misc_performance_fld',
 'misc_performance_off',
 'misc_performance_og',
 'misc_performance_recov',
 'misc_aerialduels_won',
 'misc_aerialduels_lost',
 'misc_aerialduels_won_perc',
 'keeper_penaltykicks_pka',
 'attendance']

In [189]:
l = club.columns.isin(features)
len(l)
#club.shape
club.loc[l] = moving_average(club.loc[:,l])
club

IndexError: Boolean index has wrong length: 152 instead of 222

In [408]:
past_time_features = []
for i in club.index:
    past_time_features.append(list(map(int,i.split("-"))) + list(map(int,club.loc[i, "schedule_time"].split(":"))))
for i in past_time_features:
    i[0] = i[0]-2000
past_time_features = torch.tensor(past_time_features)
future_time_features = past_time_features 



static_categorical_features = torch.tensor(club.loc[:, ['fbref_own', 'fbref_league_id']].values) # static_categorical_features
static_categorical_features

mask = club.notna()
mask
attention_mask = torch.tensor(mask.astype(int).to_numpy())
attention_mask

tensor([[17,  8, 19, 15, 30],
        [17,  8, 26, 15, 30],
        [17,  9,  9, 15, 30],
        ...,
        [23,  4, 22, 15, 30],
        [23,  4, 30, 17, 30],
        [23,  5,  7, 17, 30]])

In [73]:
from moving_average import moving_average

In [76]:
first_club = moving_average(club.loc[:,"shooting_standard_gls":"keeper_penaltykicks_pka"])

In [78]:
torch.tensor(first_club.values)

tensor([[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        ...,
        [ 1.3930, 19.5650,  7.4418,  ...,  9.0850, 53.9059,     nan],
        [ 3.6989, 20.7849,  7.7214,  ...,  5.0381, 71.9669,     nan],
        [ 2.3500, 20.8925,  6.3597,  ...,  6.0176, 67.5843,     nan]],
       dtype=torch.float64)

In [79]:
first_club

,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,shooting_standard_g_per_sot,shooting_standard_dist,shooting_standard_fk,shooting_standard_pk,shooting_standard_pkatt,...,misc_performance_2crdy,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_og,misc_performance_recov,misc_aerialduels_won,misc_aerialduels_lost,misc_aerialduels_won_perc,keeper_penaltykicks_pka
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-08,1.571848,18.253177,5.759531,30.866471,0.082747,0.267253,16.788270,0.199413,NaN,0.140762,...,0.0,12.122190,12.968719,1.010753,0.000000,51.111437,15.849462,9.356794,63.273118,NaN
2023-04-15,1.785924,22.132942,7.882698,34.686999,0.081339,0.233480,16.994233,0.099707,NaN,0.070381,...,0.0,11.562072,11.985337,1.004888,0.000000,53.060606,11.924731,9.176931,55.190029,NaN
2023-04-26,1.392962,19.565005,7.441838,37.948974,0.070674,0.186725,17.097556,0.049853,NaN,0.035191,...,0.0,9.778104,9.492669,1.503421,0.500489,55.031281,10.963832,9.085044,53.905865,NaN


In [124]:
a = new_data.dataset_team(club.loc["2023-04-08"].fbref_oppon)
b = a.loc[get_index_for_slice(a, "2023-04-08", 10)[1]:get_index_for_slice(a, "2023-04-08", 10)[0]]
c = moving_average(b)
c.iloc[-1]

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_23020/552253524.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True)
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_23020/3267728420.py:5: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['fbref_match_id', 'schedule_time'], dtype='object')
  mov_avg = df.rolling(window = window_size).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False) # compute moving average of window_size


result               0.785924
gf                   1.868035
xg                   1.234115
captain            749.026393
formation            3.000000
                      ...    
fbref_season         5.000000
fbref_league_id      9.000000
fbref_own          100.000000
fbref_oppon         17.977517
home                 0.588465
Name: 2023-04-08, Length: 150, dtype: float64

In [116]:
get_index_for_slice(a, "2023-04-08", 10)[1]

'2023-01-23'

In [87]:
def moving_average(df, window_size = 10, discount = 0.5, date = None):
    "compute the moving average with recency bias"
    weights = np.array([discount**x for x in reversed(range(window_size))]) # discount y_t-windowsize with discount**windowsize
    sum_weights = np.sum(weights)
    mov_avg = df.rolling(window = window_size).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False) # compute moving average of window_size 
    return mov_avg

In [97]:
club.loc[:"2023-04-08"]

,result,gf,xg,captain,formation,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,...,keeper_penaltykicks_pka,schedule_time,attendance,referee,fbref_season,fbref_league_id,fbref_own,fbref_oppon,fbref_match_id,home
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-12,1,0.0,0.3,847,2,0.0,6.0,2.0,33.3,0.00,...,NaN,17:30,30415.0,180,0,9,1,86,072bfc99,1
2017-08-19,1,0.0,0.2,847,5,0.0,5.0,2.0,40.0,0.00,...,0.0,15:00,31902.0,192,0,9,1,133,e487b860,0
2017-08-26,2,0.0,1.1,847,5,0.0,16.0,2.0,12.5,0.00,...,0.0,15:00,20181.0,187,0,9,1,125,7bd57325,0
2017-09-09,0,3.0,0.4,847,5,3.0,12.0,6.0,50.0,0.25,...,NaN,15:00,30381.0,191,0,9,1,11,b81fcf8d,1
2017-09-15,1,1.0,0.9,1022,5,1.0,9.0,1.0,11.1,0.11,...,0.0,20:00,10369.0,178,0,9,1,42,00836893,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-11,2,2.0,2.7,1022,0,2.0,15.0,3.0,20.0,0.13,...,0.0,15:00,36471.0,181,5,9,1,80,3f653d6f,0
2023-03-15,0,1.0,0.6,1022,0,1.0,11.0,5.0,45.5,0.09,...,NaN,19:30,30933.0,197,5,9,1,110,d23a30cc,1
2023-04-01,2,3.0,4.2,1022,0,3.0,32.0,13.0,40.6,0.06,...,NaN,15:00,31493.0,180,5,9,1,116,ead2251e,1


In [112]:
def get_index_for_slice(df, date, range_):
    a = 0
    x = []
    for i in reversed(df.loc[:date].index):
        x.append(i)
        a += 1
        if a == range_:
            break
    return x[0], x[range_ - 1]

In [113]:
get_index_for_slice(club, "2023-04-08",5)[1]

'2023-03-11'

In [154]:
a = club.loc["2023-04-08"].fbref_oppon
b = new_data.dataset_team(a)
c = moving_average(b)
e = c.loc["2023-04-08"]

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_23020/4035811546.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.end_team = random_team.merge(random_team3, "left", on = ["schedule_date"], sort = True)
/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_23020/3267728420.py:5: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['fbref_match_id', 'schedule_time'], dtype='object')
  mov_avg = df.rolling(window = window_size).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False) # compute moving average of window_size


In [148]:
for i in c.columns:
    print(i)

result
gf
xg
captain
formation
shooting_standard_gls
shooting_standard_sh
shooting_standard_sot
shooting_standard_sot_perc
shooting_standard_g_per_sh
shooting_standard_g_per_sot
shooting_standard_dist
shooting_standard_fk
shooting_standard_pk
shooting_standard_pkatt
shooting_expected_npxg
shooting_expected_npxg_per_sh
shooting_expected_g_minus_xg
shooting_expected_npg_minus_xg
keeper_performance_sota
keeper_performance_saves
keeper_performance_save_perc
keeper_performance_cs
keeper_performance_psxg
keeper_performance_psxg_plus_minus
keeper_penaltykicks_pkatt
keeper_penaltykicks_pksv
keeper_penaltykicks_pkm
keeper_launched_cmp
keeper_launched_att
keeper_launched_cmp_perc
keeper_passes_att
keeper_passes_thr
keeper_passes_launch_perc
keeper_passes_avglen
keeper_goalkicks_att
keeper_goalkicks_launch_perc
keeper_goalkicks_avglen
keeper_crosses_opp
keeper_crosses_stp
keeper_crosses_stp_perc
keeper_sweeper_number_opa
keeper_sweeper_avgdist
passing_total_cmp
passing_total_att
passing_total_cmp

In [287]:
d = moving_average(club)#.loc["2023-04-08"]
d

/var/folders/_q/xy9hjc0d0v1bds6v1r_nl1xw0000gn/T/ipykernel_23020/3267728420.py:5: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['fbref_match_id', 'schedule_time'], dtype='object')
  mov_avg = df.rolling(window = window_size).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False) # compute moving average of window_size


,result,gf,xg,captain,formation,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,...,misc_aerialduels_lost,misc_aerialduels_won_perc,keeper_penaltykicks_pka,attendance,referee,fbref_season,fbref_league_id,fbref_own,fbref_oppon,home
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-08,0.830890,1.571848,2.111535,1022.0,0.005865,1.571848,18.253177,5.759531,30.866471,0.082747,...,9.356794,63.273118,NaN,41248.512219,192.415445,5.0,9.0,1.0,87.750733,0.213099
2023-04-15,0.414467,1.785924,2.506354,1022.0,0.002933,1.785924,22.132942,7.882698,34.686999,0.081339,...,9.176931,55.190029,NaN,40691.200391,196.205279,5.0,9.0,1.0,46.813294,0.106549
2023-04-26,0.707722,1.392962,2.403226,1022.0,0.000000,1.392962,19.565005,7.441838,37.948974,0.070674,...,9.085044,53.905865,NaN,34748.235582,198.613881,5.0,9.0,1.0,63.925709,0.052786


In [292]:
d

,result,gf,xg,captain,formation,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot_perc,shooting_standard_g_per_sh,...,misc_aerialduels_lost,misc_aerialduels_won_perc,keeper_penaltykicks_pka,attendance,referee,fbref_season,fbref_league_id,fbref_own,fbref_oppon,home
schedule_date,,,,,,,,,,,,,,,,,,,,,
2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-08,0.830890,1.571848,2.111535,1022.0,0.005865,1.571848,18.253177,5.759531,30.866471,0.082747,...,9.356794,63.273118,NaN,41248.512219,192.415445,5.0,9.0,1.0,87.750733,0.213099
2023-04-15,0.414467,1.785924,2.506354,1022.0,0.002933,1.785924,22.132942,7.882698,34.686999,0.081339,...,9.176931,55.190029,NaN,40691.200391,196.205279,5.0,9.0,1.0,46.813294,0.106549
2023-04-26,0.707722,1.392962,2.403226,1022.0,0.000000,1.392962,19.565005,7.441838,37.948974,0.070674,...,9.085044,53.905865,NaN,34748.235582,198.613881,5.0,9.0,1.0,63.925709,0.052786


In [166]:
c.loc["2023-04-08",:].shape

(150,)

In [68]:
def normalize_data(df_column, x):
    df_column = torch.tensor(df_column.values)
    print(df_column)
    df_mean, df_std = torch.mean(df_column), torch.std(df_column)
    print(df_mean, df_std)
    norm = T.transforms.Normalize(df_mean, df_std)
    return norm(x)